In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [61]:
def load_xls_report(filename):
    data = pd.read_excel(filename, header=None)
    start_index = data[data.iloc[:, 0]=='Transacciones'].index[0]
    end_index = data.iloc[start_index:, 0].isna().idxmax()
    columns = data.iloc[start_index+1].tolist()
    data = data.iloc[start_index+3:end_index-1,:]
    data.columns = columns
    if('History' in filename):
        drop_columns = ['Transacción', 'Swap', 'Orden', 'Comentario', 'Tasa']
    else:
        drop_columns = ['Transacción', 'Swap', 'Orden', 'Comentario']
    data = data.drop(drop_columns, axis=1)
    data = data.set_index('Fecha/Hora')
    data.index.name = 'Fecha'
    return data

In [67]:
filename='ReportTester-4000009439.xlsx'
df = load_xls_report(filename)
df.head()

,Símbolo,Tipo,Dirección,Volumen,Precio,Comisión,Beneficio,Balance
Fecha,,,,,,,,
2022.10.03 04:10:51,GDAXI,sell,in,0.01,11845.3,-0.03,0,3999.97
2022.10.03 04:13:43,GDAXI,buy,out,0.01,11844,-0.03,0.13,4000.07
2022.10.03 04:14:23,GDAXI,sell,in,0.01,11845.7,-0.03,0,4000.04
2022.10.03 04:15:25,GDAXI,buy,out,0.01,11843.6,-0.03,0.21,4000.22
2022.10.03 04:19:57,GDAXI,sell,in,0.01,11844.7,-0.03,0,4000.19
